### Manipulation of PeaskDB de novo-assisted database search results of ETNP 2017 P2 samples LC-MS/MS data using python.

Starting with:

    PeaksDB search results (.csv) of PTM-optimized database searches
    These were all searched with 15 ppm precursor tolerance and 0.5 ppm fragement ion tolerance
    Search database included marine fungi and labrinthulomyces discovered using _de novo_ peptide sequencing
    XInteract file includes precursor intensities and protein descriptions

Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use for a different file:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the running # (e.g., 233) for another
#### 3. Update the NAAF factor calculated in *IN 6* into *IN 7*

We don't have technical duplicates here, sadly, unlike the MED4 Pro samples. I exported PeaksDB search results as `csv` files into my ETNP 2017 git repo:

In [2]:
cd /home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDB-PTMopt/

/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDB-PTMopt


In [3]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [4]:
ls 231/

'ETNP_SKQ17_PEAKSDB_161_231-100m-0.3-JA2_15ppm_DB search psm.csv'
'ETNP_SKQ17_PEAKSDB_161_231-100m-0.3-JA2_15ppm_de novo only peptides.csv'
 ETNP_SKQ17_PEAKSDB_161_231-100m-0.3-JA2_15ppm_peptides.csv
 ETNP_SKQ17_PEAKSDB_161_231-100m-0.3-JA2_15ppm_peptides.pep.xml
 ETNP_SKQ17_PEAKSDB_161_231-100m-0.3-JA2_15ppm_protein-peptides.csv
 ETNP_SKQ17_PEAKSDB_161_231-100m-0.3-JA2_15ppm_proteins.csv
 ETNP_SKQ17_PEAKSDB_161_231-100m-0.3-JA2_15ppm_proteins.fasta
 ETNP_SKQ17_PEAKSDB20_231-100m-0.3-JA2_15ppm.csv
 ETNP_SKQ17_PEAKSDB20_231-100m-0.3-JA2_15ppm_NAAF.csv
 ETNP_SKQ17_PEAKSDB20_231-100m-0.3-JA2_15ppm_NAAF_totals
 ETNP_SKQ17_PEAKSDB20_231-100m-0.3-JA2_15ppm_NAAF_totals.csv
 ETNP_SKQ17_PEAKSDB20_231-100m-0.3-JA2_15ppm_stripped_peptides.fas
 ETNP_SKQ17_PEAKSDB20_231-100m-0.3-JA2_15ppm_stripped_peptides.txt
 ETNP_SKQ17_PEAKSDB20_231-100m-0.3-JA2_15ppm_totals.csv


## 231: 100 m McLane pump filtered on 0.3 um GF-75

In [5]:
# read the CSV into a dataframe using the pandas read_csv function
peaksdbdup231 = pd.read_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDB-PTMopt/231/ETNP_SKQ17_PEAKSDB_161_231-100m-0.3-JA2_15ppm_peptides.csv")

# remove redundant rows
peaksdb231 = pd.DataFrame.drop_duplicates(peaksdbdup231)

print(peaksdb231.columns)

columns = ['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area', 'Fraction', 'Scan', 'Source File',
       '#Spec', '#Spec', 'Accession', 'PTM',
       'AScore']

peaksdb231.columns = columns

print("# redundant peaksdb peptides in combined dataframe", len(peaksdbdup231))
print("# nonredundant peaksdb peptides in combined dataframe", len(peaksdb231))

#look at the dataframe
peaksdb231.head()

Index(['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area 231_JA2_100m_0.3um_SKQ17_P2', 'Fraction', 'Scan', 'Source File',
       '#Spec', '#Spec 231_JA2_100m_0.3um_SKQ17_P2', 'Accession', 'PTM',
       'AScore'],
      dtype='object')
# redundant peaksdb peptides in combined dataframe 1299
# nonredundant peaksdb peptides in combined dataframe 1299


Peptide  -10lgP       Mass  Length  ppm        m/z     RT  \
0   IVVGGPYSSVSDAASSLDSSQK   88.11  2153.0488      22  2.7  1077.5345  71.78   
1   IVVGGPYSSVSDAASVLDGSQK   88.10  2135.0745      22  1.6  1068.5463  82.14   
2  AIQQQIENPLAQQILSGELVPGK   87.31  2473.3540      23  3.6  1237.6887  98.28   
3    QAVSADSSGSFIGGAELASLK   83.59  1993.9956      21  1.0   998.0060  81.22   
4      LPQVEGTGGDVQPSQDLVR   83.40  1994.0068      19  1.9   998.0126  62.23   

          Area  Fraction   Scan                              Source File  \
0   25300000.0         1  23179  20170410_ETNP-231-100m-0.3um-JA2_01.raw   
1   20600000.0         1  28172  20170410_ETNP-231-100m-0.3um-JA2_01.raw   
2   25900000.0         1  34897  20170410_ETNP-231-100m-0.3um-JA2_01.raw   
3    6280000.0         1  27723  20170410_ETNP-231-100m-0.3um-JA2_01.raw   
4  237000000.0         1  19110  20170410_ETNP-231-100m-0.3um-JA2_01.raw   

   #Spec  #Spec                                          Accession  PTM AScore  
0      6      6      ETNP_160m_PROKKA_83311:ETNP_110m_PROKKA_45826  NaN    NaN  
1      7      7  ETNP_120m_PROKKA_126325:ETNP_90m_PROKKA_27133:...  NaN    NaN  
2      6      6                                        gi|54036848  NaN    NaN  
3      7      7  ETNP_120m_PROKKA_239025:ETNP_120m_free_PROKKA_...  NaN    NaN  
4     13     13                                        gi|54036848  NaN    NaN

In [7]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaksdb231['A'] = peaksdb231['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaksdb231['C'] = peaksdb231['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaksdb231['D'] = peaksdb231['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaksdb231['E'] = peaksdb231['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaksdb231['F'] = peaksdb231['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaksdb231['G'] = peaksdb231['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaksdb231['H'] = peaksdb231['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaksdb231 output, there will be no isoleucines (they're lumped in with leucines)
peaksdb231['I'] = peaksdb231['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaksdb231['K'] = peaksdb231['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaksdb231['L'] = peaksdb231['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaksdb231['M'] = peaksdb231['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaksdb231['N'] = peaksdb231['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaksdb231['P'] = peaksdb231['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaksdb231['Q'] = peaksdb231['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaksdb231['R'] = peaksdb231['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaksdb231['S'] = peaksdb231['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaksdb231['T'] = peaksdb231['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaksdb231['V'] = peaksdb231['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaksdb231['W'] = peaksdb231['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaksdb231['Y'] = peaksdb231['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaksdb231['c-carb'] = peaksdb231['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaksdb231['m-oxid'] = peaksdb231['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaksdb231['n-deam'] = peaksdb231['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a count function to enumerate the # of deamidated Q's in each peptide
peaksdb231['q-deam'] = peaksdb231['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# use a count function to enumerate the # of hydroxylated K's in each peptide
peaksdb231['k-hydr'] = peaksdb231['Peptide'].apply(lambda x: x.count('K(+15.99)'))

# use a count function to enumerate the # of hydroxylated P's in each peptide
#peaksdb231['p-hydr'] = peaksdb231['Peptide'].apply(lambda x: x.count('P(+15.99)'))

# use a count function to enumerate the # of methylated R's in each peptide
peaksdb231['r-meth'] = peaksdb231['Peptide'].apply(lambda x: x.count('R(+14.02)'))

# create a column with 'stripped' peptide sequences using strip
peaksdb231['stripped peptide'] = peaksdb231['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaksdb231['stripped length'] = peaksdb231['stripped peptide'].apply(len)

peaksdb231['NAAF num.'] = peaksdb231['Area'] / peaksdb231['stripped length']

# total the number of modifications in sequence
peaksdb231['ptm-total'] = peaksdb231['c-carb'] + peaksdb231['m-oxid'] + peaksdb231['n-deam'] + peaksdb231['q-deam'] + peaksdb231['k-hydr'] + peaksdb231['r-meth']

# turn all isoleucines into leucines
# this helps later in comparing Unipept peptides to PeaksDB and Comet ones
peaksdb231['stripped I-L']= peaksdb231['stripped peptide'].str.replace('I','L')

# write modified dataframe to new txt file
peaksdb231.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDB-PTMopt/231/ETNP_SKQ17_PEAKSDB20_231-100m-0.3-JA2_15ppm.csv")

# check out the results
peaksdb231.head()

Peptide  -10lgP       Mass  Length  ppm        m/z     RT  \
0   IVVGGPYSSVSDAASSLDSSQK   88.11  2153.0488      22  2.7  1077.5345  71.78   
1   IVVGGPYSSVSDAASVLDGSQK   88.10  2135.0745      22  1.6  1068.5463  82.14   
2  AIQQQIENPLAQQILSGELVPGK   87.31  2473.3540      23  3.6  1237.6887  98.28   
3    QAVSADSSGSFIGGAELASLK   83.59  1993.9956      21  1.0   998.0060  81.22   
4      LPQVEGTGGDVQPSQDLVR   83.40  1994.0068      19  1.9   998.0126  62.23   

          Area  Fraction   Scan  ... m-oxid  n-deam  q-deam k-hydr r-meth  \
0   25300000.0         1  23179  ...      0       0       0      0      0   
1   20600000.0         1  28172  ...      0       0       0      0      0   
2   25900000.0         1  34897  ...      0       0       0      0      0   
3    6280000.0         1  27723  ...      0       0       0      0      0   
4  237000000.0         1  19110  ...      0       0       0      0      0   

          stripped peptide  stripped length     NAAF num.  ptm-total  \
0   IVVGGPYSSVSDAASSLDSSQK               22  1.150000e+06          0   
1   IVVGGPYSSVSDAASVLDGSQK               22  9.363636e+05          0   
2  AIQQQIENPLAQQILSGELVPGK               23  1.126087e+06          0   
3    QAVSADSSGSFIGGAELASLK               21  2.990476e+05          0   
4      LPQVEGTGGDVQPSQDLVR               19  1.247368e+07          0   

              stripped I-L  
0   LVVGGPYSSVSDAASSLDSSQK  
1   LVVGGPYSSVSDAASVLDGSQK  
2  ALQQQLENPLAQQLLSGELVPGK  
3    QAVSADSSGSFLGGAELASLK  
4      LPQVEGTGGDVQPSQDLVR  

[5 rows x 47 columns]

### Exporting txt files of stripped peptides at confidence cutoffs:

In [6]:
# keep only peptide column peptides >20 -10lgP
pep231moddup = peaksdb231[["Peptide"]]

# keep only the stripped peptide column >20 -10lgP
# this is what we'll use for UniPept input, etc
pep231dup = peaksdb231[["stripped peptide"]]

# deduplicate both of these lists
pep231mod = pep231moddup.drop_duplicates()
pep231 = pep231dup.drop_duplicates()

# print out the #s of modified and stripped peptides, deduplicated and not

print("Total modified PeaksDB peptides in 231:", len(pep231moddup))
print("Deduplicated modified PeaksDB peptides in 231:", len(pep231mod))
print("Total stripped PeaksDB peptides in 231:", len(pep231dup))
print("Deduplicated stripped PeaksDB peptides in 231:", len(pep231))

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing
pep231.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDB-PTMopt/231/ETNP_SKQ17_PEAKSDB20_231-100m-0.3-JA2_15ppm_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 
!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDB-PTMopt/231/ETNP_SKQ17_PEAKSDB20_231-100m-0.3-JA2_15ppm_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDB-PTMopt/231/ETNP_SKQ17_PEAKSDB20_231-100m-0.3-JA2_15ppm_stripped_peptides.fas

# look at the stripped peptides
pep231.head()

Total modified PeaksDB peptides in 231: 1299
Deduplicated modified PeaksDB peptides in 231: 1299
Total stripped PeaksDB peptides in 231: 1299
Deduplicated stripped PeaksDB peptides in 231: 1278


stripped peptide
0   IVVGGPYSSVSDAASSLDSSQK
1   IVVGGPYSSVSDAASVLDGSQK
2  AIQQQIENPLAQQILSGELVPGK
3    QAVSADSSGSFIGGAELASLK
4      LPQVEGTGGDVQPSQDLVR

## NAAF correction and exporting files with AA and PTM totals:

In [7]:
# made a new dataframe that contains the sums of certain columns in the modified
# peptide dataframe above 

index = ['sample total']

data = {'A': peaksdb231['A'].sum(),
        'C': peaksdb231['C'].sum(),
        'D': peaksdb231['D'].sum(),
        'E': peaksdb231['E'].sum(),
        'F': peaksdb231['F'].sum(),
        'G': peaksdb231['G'].sum(),
        'H': peaksdb231['H'].sum(),
        'I': peaksdb231['I'].sum(),
        'K': peaksdb231['K'].sum(),
        'L': peaksdb231['L'].sum(),
        'M': peaksdb231['M'].sum(),
        'N': peaksdb231['N'].sum(),
        'P': peaksdb231['P'].sum(),
        'Q': peaksdb231['Q'].sum(),
        'R': peaksdb231['R'].sum(),
        'S': peaksdb231['S'].sum(),
        'T': peaksdb231['T'].sum(),
        'V': peaksdb231['V'].sum(),
        'W': peaksdb231['W'].sum(),
        'Y': peaksdb231['Y'].sum(),
        'c-carb': peaksdb231['c-carb'].sum(),
        'm-oxid': peaksdb231['m-oxid'].sum(),
        'n-deam': peaksdb231['n-deam'].sum(),
        'q-deam': peaksdb231['q-deam'].sum(),
        'k-hydr': peaksdb231['k-hydr'].sum(),
        'r-meth': peaksdb231['r-meth'].sum(),
        'Total area': peaksdb231['Area'].sum(),
        'Total length': peaksdb231['stripped length'].sum()
       }

totalpeaksdb231 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', 'm-oxid', 'n-deam', 'q-deam', 'k-hydr', 'r-meth', 'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaksdb231['% C w/ carb'] = totalpeaksdb231['c-carb'] / totalpeaksdb231['C'] 

# calculate percentage of M's that are oxidized
totalpeaksdb231['% M w/ oxid'] = totalpeaksdb231['m-oxid'] / totalpeaksdb231['M'] 

# calculate percentage of N's that are deamidated
totalpeaksdb231['% N w/ deam'] = totalpeaksdb231['n-deam'] / totalpeaksdb231['N'] 

# calculate percentage of Q's that are deamidated
totalpeaksdb231['% Q w/ deam'] = totalpeaksdb231['q-deam'] / totalpeaksdb231['Q'] 

# calculate percentage of K's that are hydroxylated
totalpeaksdb231['% K w/ hydr'] = totalpeaksdb231['k-hydr'] / totalpeaksdb231['K'] 

# calculate percentage of P's that are hydroxylated
#totalpeaksdb231['% P w/ hydr'] = totalpeaksdb231['p-hydr'] / totalpeaksdb231['P'] 

# calculate percentage of R's that are methylated
totalpeaksdb231['% R w/ meth'] = totalpeaksdb231['r-meth'] / totalpeaksdb231['R'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaksdb231['NAAF denom.'] = totalpeaksdb231['Total area'] / totalpeaksdb231['Total length']

# write modified dataframe to new txt file
totalpeaksdb231.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDB-PTMopt/231/ETNP_SKQ17_PEAKSDB20_231-100m-0.3-JA2_15ppm_totals.csv")

totalpeaksdb231.head()

A   C     D     E    F     G   H    I    K     L  ...  \
sample total  1786  77  1133  1308  590  1835  70  894  789  1443  ...   

              r-meth    Total area  Total length  % C w/ carb  % M w/ oxid  \
sample total       6  3.867077e+09         16576          1.0     0.858974   

              % N w/ deam  % Q w/ deam  % K w/ hydr  % R w/ meth  \
sample total     0.017632     0.019332     0.010139     0.009077   

                NAAF denom.  
sample total  233293.736728  

[1 rows x 35 columns]

In [8]:
# use the calculated NAAF factor (in totalpeaksdb231 dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

NAAF20 = 233293.736728

# use NAAF >XCorr 3 to get NAAF
peaksdb231['NAAF factor'] = (peaksdb231['NAAF num.'])/NAAF20

# make a dataframe that contains only what we need: sequences, AAs, PTMs
peaksdb231_NAAF = peaksdb231[['stripped peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', \
                                'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', 'm-oxid', 'n-deam', \
                                'q-deam', 'k-hydr', 'r-meth']].copy()

# multiply the NAAF20 factor by the AAs to normalize its abundance by peak area and peptide length

peaksdb231_NAAF['A-NAAF20'] = peaksdb231_NAAF['A'] * peaksdb231['NAAF factor']
peaksdb231_NAAF['C-NAAF20'] = peaksdb231_NAAF['C'] * peaksdb231['NAAF factor']
peaksdb231_NAAF['D-NAAF20'] = peaksdb231_NAAF['D'] * peaksdb231['NAAF factor']
peaksdb231_NAAF['E-NAAF20'] = peaksdb231_NAAF['E'] * peaksdb231['NAAF factor']
peaksdb231_NAAF['F-NAAF20'] = peaksdb231_NAAF['F'] * peaksdb231['NAAF factor']
peaksdb231_NAAF['G-NAAF20'] = peaksdb231_NAAF['G'] * peaksdb231['NAAF factor']
peaksdb231_NAAF['H-NAAF20'] = peaksdb231_NAAF['H'] * peaksdb231['NAAF factor']
peaksdb231_NAAF['I-NAAF20'] = peaksdb231_NAAF['I'] * peaksdb231['NAAF factor']
peaksdb231_NAAF['K-NAAF20'] = peaksdb231_NAAF['K'] * peaksdb231['NAAF factor']
peaksdb231_NAAF['L-NAAF20'] = peaksdb231_NAAF['L'] * peaksdb231['NAAF factor']
peaksdb231_NAAF['M-NAAF20'] = peaksdb231_NAAF['M'] * peaksdb231['NAAF factor']
peaksdb231_NAAF['N-NAAF20'] = peaksdb231_NAAF['N'] * peaksdb231['NAAF factor']
peaksdb231_NAAF['P-NAAF20'] = peaksdb231_NAAF['P'] * peaksdb231['NAAF factor']
peaksdb231_NAAF['Q-NAAF20'] = peaksdb231_NAAF['Q'] * peaksdb231['NAAF factor']
peaksdb231_NAAF['R-NAAF20'] = peaksdb231_NAAF['R'] * peaksdb231['NAAF factor']
peaksdb231_NAAF['S-NAAF20'] = peaksdb231_NAAF['S'] * peaksdb231['NAAF factor']
peaksdb231_NAAF['T-NAAF20'] = peaksdb231_NAAF['T'] * peaksdb231['NAAF factor']
peaksdb231_NAAF['V-NAAF20'] = peaksdb231_NAAF['V'] * peaksdb231['NAAF factor']
peaksdb231_NAAF['W-NAAF20'] = peaksdb231_NAAF['W'] * peaksdb231['NAAF factor']
peaksdb231_NAAF['Y-NAAF20'] = peaksdb231_NAAF['Y'] * peaksdb231['NAAF factor']

# multiply the NAAF20 factor by the PTMs normalize its abundance by peak area and peptide length

peaksdb231_NAAF['ccarb-NAAF20'] = peaksdb231_NAAF['c-carb'] * peaksdb231_NAAF['NAAF factor']
peaksdb231_NAAF['moxid-NAAF20'] = peaksdb231_NAAF['m-oxid'] * peaksdb231_NAAF['NAAF factor']
peaksdb231_NAAF['ndeam-NAAF20'] = peaksdb231_NAAF['n-deam'] * peaksdb231_NAAF['NAAF factor']
peaksdb231_NAAF['qdeam-NAAF20'] = peaksdb231_NAAF['q-deam'] * peaksdb231_NAAF['NAAF factor']
peaksdb231_NAAF['khydr-NAAF20'] = peaksdb231_NAAF['k-hydr'] * peaksdb231_NAAF['NAAF factor']
peaksdb231_NAAF['rmeth-NAAF20'] = peaksdb231_NAAF['r-meth'] * peaksdb231_NAAF['NAAF factor']

# write the dataframe to a new csv
peaksdb231_NAAF.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDB-PTMopt/231/ETNP_SKQ17_PEAKSDB20_231-100m-0.3-JA2_15ppm_NAAF.csv")

peaksdb231_NAAF.head()

stripped peptide  NAAF factor  A  C  D  E  F  G  H  I  ...  \
0   IVVGGPYSSVSDAASSLDSSQK     4.929408  2  0  2  0  0  2  0  1  ...   
1   IVVGGPYSSVSDAASVLDGSQK     4.013668  2  0  2  0  0  3  0  1  ...   
2  AIQQQIENPLAQQILSGELVPGK     4.826906  2  0  0  2  0  2  0  3  ...   
3    QAVSADSSGSFIGGAELASLK     1.281850  4  0  1  1  1  3  0  1  ...   
4      LPQVEGTGGDVQPSQDLVR    53.467720  0  0  2  1  0  3  0  0  ...   

   T-NAAF20    V-NAAF20  W-NAAF20  Y-NAAF20  ccarb-NAAF20  moxid-NAAF20  \
0   0.00000   14.788224       0.0  4.929408           0.0           0.0   
1   0.00000   16.054673       0.0  4.013668           0.0           0.0   
2   0.00000    4.826906       0.0  0.000000           0.0           0.0   
3   0.00000    1.281850       0.0  0.000000           0.0           0.0   
4  53.46772  160.403160       0.0  0.000000           0.0           0.0   

   ndeam-NAAF20  qdeam-NAAF20  khydr-NAAF20  rmeth-NAAF20  
0           0.0           0.0           0.0           0.0  
1           0.0           0.0           0.0           0.0  
2           0.0           0.0           0.0           0.0  
3           0.0           0.0           0.0           0.0  
4           0.0           0.0           0.0           0.0  

[5 rows x 54 columns]

In [9]:
# made a new dataframe that contains the sums of NAAF normalized AAs for peaksdb231 peaksdb results
# also contains the sums of the NAAF-corrected PTMs occurances for each affected residue

index = ['sample total']

data = {'NAAF': peaksdb231_NAAF['NAAF factor'].sum(),
        'A-NAAF': peaksdb231_NAAF['A-NAAF20'].sum(),
        'C-NAAF': peaksdb231_NAAF['C-NAAF20'].sum(),
        'D-NAAF': peaksdb231_NAAF['D-NAAF20'].sum(),
        'E-NAAF': peaksdb231_NAAF['E-NAAF20'].sum(),
        'F-NAAF': peaksdb231_NAAF['F-NAAF20'].sum(),
        'G-NAAF': peaksdb231_NAAF['G-NAAF20'].sum(),
        'H-NAAF': peaksdb231_NAAF['H-NAAF20'].sum(),
        'I-NAAF': peaksdb231_NAAF['I-NAAF20'].sum(),
        'K-NAAF': peaksdb231_NAAF['K-NAAF20'].sum(),
        'L-NAAF': peaksdb231_NAAF['L-NAAF20'].sum(),
        'M-NAAF': peaksdb231_NAAF['M-NAAF20'].sum(),
        'N-NAAF': peaksdb231_NAAF['N-NAAF20'].sum(),
        'P-NAAF': peaksdb231_NAAF['P-NAAF20'].sum(),
        'Q-NAAF': peaksdb231_NAAF['Q-NAAF20'].sum(),
        'R-NAAF': peaksdb231_NAAF['R-NAAF20'].sum(),
        'S-NAAF': peaksdb231_NAAF['S-NAAF20'].sum(),
        'T-NAAF': peaksdb231_NAAF['T-NAAF20'].sum(),
        'V-NAAF': peaksdb231_NAAF['V-NAAF20'].sum(),
        'W-NAAF': peaksdb231_NAAF['W-NAAF20'].sum(),
        'Y-NAAF': peaksdb231_NAAF['Y-NAAF20'].sum(),
        'C-carb-NAAF': peaksdb231_NAAF['ccarb-NAAF20'].sum(),
        'M-oxid-NAAF': peaksdb231_NAAF['moxid-NAAF20'].sum(),
        'N-deam-NAAF': peaksdb231_NAAF['ndeam-NAAF20'].sum(),
        'Q-deam-NAAF': peaksdb231_NAAF['qdeam-NAAF20'].sum(),
        'K-hydr-NAAF': peaksdb231_NAAF['khydr-NAAF20'].sum(),
        'R-meth-NAAF': peaksdb231_NAAF['rmeth-NAAF20'].sum()
       }

totalpeaksdb231_NAAF = pd.DataFrame(data, columns=['NAAF', 'A-NAAF', 'C-NAAF', 'D-NAAF', 'E-NAAF', 'F-NAAF', \
                                                   'G-NAAF', 'H-NAAF', 'I-NAAF','K-NAAF', 'L-NAAF', 'M-NAAF', \
                                                   'N-NAAF', 'P-NAAF', 'Q-NAAF', 'R-NAAF', 'S-NAAF', \
                                                   'T-NAAF', 'V-NAAF', 'W-NAAF', 'Y-NAAF', 'C-carb-NAAF', \
                                                   'M-oxid-NAAF', 'N-deam-NAAF', 'Q-deam-NAAF', 'K-hydr-NAAF',\
                                                   'R-meth-NAAF'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q, K, and Rs


totalpeaksdb231_NAAF['% C w/ carb. NAAF'] = totalpeaksdb231_NAAF['C-carb-NAAF'] / totalpeaksdb231_NAAF['C-NAAF']
totalpeaksdb231_NAAF['% M w/ oxid. NAAF'] = totalpeaksdb231_NAAF['M-oxid-NAAF'] / totalpeaksdb231_NAAF['M-NAAF']
totalpeaksdb231_NAAF['% N w/ deam. NAAF'] = totalpeaksdb231_NAAF['N-deam-NAAF'] / totalpeaksdb231_NAAF['N-NAAF']
totalpeaksdb231_NAAF['% Q w/ deam. NAAF'] = totalpeaksdb231_NAAF['Q-deam-NAAF'] / totalpeaksdb231_NAAF['Q-NAAF']
totalpeaksdb231_NAAF['% K w/ hydr. NAAF'] = totalpeaksdb231_NAAF['K-hydr-NAAF'] / totalpeaksdb231_NAAF['K-NAAF']
totalpeaksdb231_NAAF['% R w/ meth. NAAF'] = totalpeaksdb231_NAAF['R-meth-NAAF'] / totalpeaksdb231_NAAF['R-NAAF']

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaksdb231_NAAF['NAAF check'] = totalpeaksdb231_NAAF['NAAF'] / 233293.736728

# write modified dataframe to new txt file, same name + totals
totalpeaksdb231_NAAF.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDB-PTMopt/231/ETNP_SKQ17_PEAKSDB20_231-100m-0.3-JA2_15ppm_NAAF_totals")

totalpeaksdb231_NAAF.head()

NAAF       A-NAAF     C-NAAF       D-NAAF       E-NAAF  \
sample total  1502.41746  2078.244989  35.328251  1117.903003  1140.395347   

                  F-NAAF       G-NAAF     H-NAAF       I-NAAF      K-NAAF  \
sample total  301.460433  1303.225805  39.415329  1076.560933  556.854934   

              ...  Q-deam-NAAF  K-hydr-NAAF  R-meth-NAAF  % C w/ carb. NAAF  \
sample total  ...     1.506736      9.37155    11.166295                1.0   

              % M w/ oxid. NAAF  % N w/ deam. NAAF  % Q w/ deam. NAAF  \
sample total           0.854814           0.001211           0.002994   

              % K w/ hydr. NAAF  % R w/ meth. NAAF  NAAF check  
sample total           0.016829           0.010988     0.00644  

[1 rows x 34 columns]

### Using BioPython to query peptide sequences

I installed the BioPython package using `pip install biopython`. All instructions and information [here](https://www.tutorialspoint.com/biopython/index.htm). 

GitHub project: https://github.com/biopython/biopython

I'm relying on the ProtParam module to parse sequences for relative AA composition, instability, secondary structure, instability, and hydrophobicity. You can read more about that module and the studies the indecies are derived from here:

https://biopython.org/wiki/ProtParam

In [ ]:
# Bio.SeqIO is the standard Sequence Input/Output interface for BioPython 1.43 and later
# Bio.SeqIO provides a simple uniform interface to input and output assorted sequence file formats.
# (including multiple sequence alignments), but will only deal with sequences as SeqRecord objects

# for accepted file formats see https://biopython.org/wiki/SeqIO

from Bio import SeqIO
#for seq_record in SeqIO.parse("/home/millieginty/Documents/git-repos/2017-etnp/data/MED4/MED2_tryp_1raw_db_peptides_nmod.fasta", "fasta"):
    #print(seq_record.id)
    #print(repr(seq_record.seq))
    #print(len(seq_record))
    
# I commented the print functions out so the output doesn't take up too much space. 

In [ ]:
# seeing what the ProtParam module can do with a single protein sequence:

from Bio.SeqUtils.ProtParam import ProteinAnalysis

test_seq = "MAEGEITTFTALTEKFNLPPGNYKKPKLLYCSNGGHFLRILPDGTVDGTRDRSDQHIQLQLSAESVGEVYIKSTETGQYLAMDTSGLLYGSQTPSEECLFLERLEENHYNTYTSKKHAEKNWFVGLKKNGSCKRGPRTHYGQKAILFLPLPV"

analysed_seq = ProteinAnalysis(test_seq)
print("molecular weight of seq =", analysed_seq.molecular_weight())

# calculates the aromaticity value of a protein according to Lobry & Gautier (1994, Nucleic Acids Res., 22, 3174-3180). 
# it's simply the relative frequency of Phe+Trp+Tyr.

analysed_seq.aromaticity()
print("aromaticity of seq =", analysed_seq.aromaticity())

# secondary_structure_fraction:
# this methods returns a list of the fraction of amino acids which tend to be in helix, turn or sheet. 
# AAs in helix: V, I, Y, F, W, L
# AAs in turn: N, P, G, S
# AAs in sheet: E, M, A, L
# the returned list contains 3 values: [Helix, Turn, Sheet]

analysed_seq.secondary_structure_fraction()
print("frac in H T S =", analysed_seq.secondary_structure_fraction())

# the instability index, an implementation of the method of Guruprasad et al. (1990, Protein Engineering, 4, 155-161).
# this method tests a protein for stability. 
# any value above 40 means the protein is unstable (=has a short half life)
# NOT SURE WHAT THIS MEANS FOR PEPTIDES, BUT WE COULD DO THIS FOR PROTEINS

analysed_seq.instability_index()
print("instability =", analysed_seq.instability_index())

# count_amino_acids will do just that, and get_amino_acids_percent will return %'s for each AA across the sequence. 
analysed_seq.get_amino_acids_percent()

# taking the returned dictionary and converting to a dataframe

aadict = analysed_seq.get_amino_acids_percent()
aadf = pd.DataFrame(list(aadict.items()),columns = ['residue','% occurance']) 

aadf.head()

In [ ]:
# use SeqIO and a loop to apply count_amino_acids to each sequence in the file
# aatot will give us the total number of each residue in the entire sample output

import collections
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis

all_aas = collections.defaultdict(int)
for record in SeqIO.parse("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-231-100m-0.3-JA2_DN50_stripped_peptides.fas", "fasta"):
    x = ProteinAnalysis(str(record.seq))
    #print(record.id, x.count_amino_acids())
    for aa, count in x.count_amino_acids().items():
        all_aas[aa] += count        
        
# made a dataframe for amino acid total counts        
data = (all_aas)
aatot = pd.DataFrame(data, index = ['sample sequence total'])
aatot.head()

In [ ]:
from pandas import Series, DataFrame

with open('/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-231-100m-0.3-JA2_DN50_stripped_peptides.fas') as fasta_file:  # Will close handle cleanly
    identifiers = []
    lengths = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        identifiers.append(seq_record.id)
        lengths.append(len(seq_record.seq))
        
        
#converting lists to pandas Series    
s1 = Series(identifiers, name='ID')
s2 = Series(lengths, name='length')

#Gathering Series into a pandas DataFrame and rename index as ID column
idseq = DataFrame(dict(ID=s1, length=s2)).set_index(['ID'])

idseq.head()

In [ ]:
from pandas import Series, DataFrame



with open('/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-231-100m-0.3-JA2_DN50_stripped_peptides.fas) as fasta_file:  # Will close handle cleanly
    identifiers = []
    lengths = []
    aa = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        identifiers.append(seq_record.id)
        lengths.append(len(seq_record.seq))
        aa.count_amino_acids(seq_record.seq)
        
        
#converting lists to pandas Series    
s1 = Series(identifiers, name='ID')
s2 = Series(lengths, name='length')
s3 = Series(aa, name='AAs')

#Gathering Series into a pandas DataFrame and rename index as ID column
idseq = DataFrame(dict(ID=s1, length=s2, AAs=s3)).set_index(['ID'])

idseq.head()

In [ ]:
from Bio import SeqIO
from Bio.SeqUtils import ProtParam

handle = open("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-231-100m-0.3-JA2_DN50_stripped_peptides.fas") 
for record in SeqIO.parse(handle, "fasta"): 
    seq = str(record.seq)
    X = ProtParam.ProteinAnalysis(seq)
    print(X.count_amino_acids()) 
    #print X.get_amino_acids_percent() 
    #print X.molecular_weight() 
    #print X.aromaticity() 
    #print X.instability_index() 
    #print X.flexibility() 
    #print X.isoelectric_point() 
    #print X.secondary_structure_fraction()
    
# made a data series from the count_amino_acids function
# aacount = {X.count_amino_acids()}

# made a pandas dataframe from the series generated above
# aacount = pd.DataFrame(list(data.items()),columns = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']) 

aacount = pd.DataFrame(X.count_amino_acids(), index=[0])

# look at new dataframe

# aacount.head()